<a href="https://colab.research.google.com/github/DamnCrab/Real-ESRGAN-Inference-Demo/blob/main/Real_ESRGAN_Inference_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-ESRGAN Inference Demo

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2107.10833)
[![GitHub Stars](https://img.shields.io/github/stars/xinntao/Real-ESRGAN?style=social)](https://github.com/xinntao/Real-ESRGAN)
[![download](https://img.shields.io/github/downloads/xinntao/Real-ESRGAN/total.svg)](https://github.com/xinntao/Real-ESRGAN/releases)

This is a **Practical Image Restoration Demo** of our paper [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833).
We extend the powerful ESRGAN to a practical restoration application (namely, Real-ESRGAN), which is trained with pure synthetic data. <br>
The following figure shows some real-life examples.

<img src="https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/assets/teaser.jpg" width="100%">

We provide a pretrained model (*RealESRGAN_x4plus.pth*) with upsampling X4.<br>
**Note that RealESRGAN may still fail in some cases as the real-world degradations are really too complex.**<br>
Moreover, it **may not** perform well on **human faces, text**, *etc*, which will be optimized later.
<br>

You can also find a **Portable Windows/Linux/MacOS executable files for Intel/AMD/Nvidia GPU.** in our [GitHub repo](https://github.com/xinntao/Real-ESRGAN). <br>
This executable file is **portable** and includes all the binaries and models required. No CUDA or PyTorch environment is needed.<br>
This executable file is based on the wonderful [Tencent/ncnn](https://github.com/Tencent/ncnn) and [realsr-ncnn-vulkan](https://github.com/nihui/realsr-ncnn-vulkan) by [nihui](https://github.com/nihui).

# 1. Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the envrironment, and download the pre-trained model.

1.1 显卡检测和系统资源监控

In [ ]:
#检测使用的显卡
! /opt/bin/nvidia-smi
! nvcc --version

In [7]:
#注册https://wandb.ai/authorize 填api 获得实时监控网址
!pip install wandb
import wandb
wandb.init()

     |████████████████████████████████| 1.8 MB 8.1 MB/s 
     |████████████████████████████████| 97 kB 8.9 MB/s 
     |████████████████████████████████| 133 kB 61.4 MB/s 
     |████████████████████████████████| 170 kB 60.0 MB/s 
     |████████████████████████████████| 138 kB 74.1 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=208c849ec97d5ebeddadaab247abc1f3fd3d2a1e82fd61de9417e136c1f451f0
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=5b8a0ab43690256fc9a54da78101674507105f627a9a3fa869cfebfd7a482c38
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
  Attempting uninstall: urlli

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


1.2 完整FFmpeg安装 （可跳过 安装需要30min-40min 安装后可以使用硬件加速和10bit


In [ ]:
!yes '' | sudo add-apt-repository ppa:savoury1/graphics
!yes '' | sudo add-apt-repository ppa:savoury1/multimedia
!yes '' | sudo add-apt-repository ppa:savoury1/multimedia
!yes '' | sudo add-apt-repository ppa:savoury1/ffmpeg4
!sudo apt-get update
#!sudo apt-get upgrade && sudo apt-get dist-upgrade
!sudo apt-get install ffmpeg

In [ ]:
#安装成功应为ffmpeg version 4.4-1ubuntu0~18.04.sav0.1
%shell ffmpeg -hwaccels -h 

1.3 Real-ESRGAN下载

In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
%cd /content
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
# !pip install -r requirements.txt
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

# 2. Upload Images

Upload the images to be processed by Real-ESRGAN

2.1 网页上传或者Drive上传 （建议Drive 网页上传经常会很慢

In [ ]:
#网页上传方式
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)



In [ ]:
#Drive上传方式 需要授权
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from google.colab import files
import shutil
upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

filename='3.mp4' #填文件名########################################################################

dst_path1 = os.path.join("/content/drive/MyDrive/", filename)
shutil.copy(dst_path1,"/content/Real-ESRGAN/upload/")

2.2检测输入文件FPS

In [ ]:

tmp_path = os.path.join("/content/Real-ESRGAN/upload/", filename)
import cv2

video = cv2.VideoCapture(tmp_path)

fps = video.get(cv2.CAP_PROP_FPS)
print("帧率: {0}".format(fps))
frame_num = video.get(cv2.CAP_PROP_FRAME_COUNT)
print("帧数: {0}".format(frame_num))

#av1编码有bug 检测不到



In [ ]:
#%shell ffmpeg -i '/content/Real-ESRGAN/upload/{filename}' -codec copy -map 0 -f segment -segment_list out.csv -segment_frames 1000,2000,3000,4000,5000 out%03d.mkv

In [ ]:
#!apt-get install nodejs-dev node-gyp libssl1.0-dev
#!apt-get install npm
#!$ npm install --global ffmpeg-progressbar-cli


In [ ]:
#! npm install --global ffmpeg-progressbar-cli

In [ ]:
#%shell ffprobe -v error -show_streams -print_format json '/content/Real-ESRGAN/upload/{filename}'  

2.3 拆帧

In [ ]:
%shell rm -rf '/content/Real-ESRGAN/frame'
%shell mkdir -p '/content/Real-ESRGAN/frame'
#%shell ffmpeg -hwaccel_output_format cuda -i '/content/Real-ESRGAN/upload/{filename}' '/content/Real-ESRGAN/frame/%08d.png'
%shell ffmpeg -hwaccel nvdec -i '/content/Real-ESRGAN/upload/{filename}' '/content/Real-ESRGAN/frame/%08d.png'


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fc0a39ff090>> (for pre_run_cell):


Exception: ignored

ffmpeg version 4.4-1ubuntu0~18.04.sav0.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version='1ubuntu0~18.04.sav0.1' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --

# 3. 超分


In [ ]:
# if it is out of memory, try to use the `--tile` option 爆显存调tile 现值100
!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input frame --scale 4 --half --tile 100

# 4. 压制 使用者佬参数

In [ ]:
%shell mkdir -p '/content/Real-ESRGAN/output'

In [14]:
filename_out='output.mkv'                         #输出文件名
output_pth = os.path.join("/content/Real-ESRGAN/output", filename_out) #输出文件夹
#x264 x265参数设置
crf=16 #[-12-51]
preset='slower' #ultrafast、superfast、veryfast、faster、fast、medium、slow、slower、veryslow、placebo
#nvenc 参数设置
preset_nvenc=4
cq=16
#
#-cq                <float>      E..V....... Set target quality level (0 to 51, 0 means automatic) for constant quality mode in VBR rate control (from 0 to 51) (default 0)
#-preset            <int>        E..V....... Set the encoding preset (from 0 to 11) (default medium)
#     default         0            E..V....... 
#     slow            1            E..V....... hq 2 passes
#     fast            3            E..V....... hp 1 pass
#     hp              4            E..V....... 
#     hq              5            E..V....... 
#     bd              6            E..V....... 
#     ll              7            E..V....... low latency
#     llhq            8            E..V....... low latency hq
#     llhp            9            E..V....... low latency hp
#     lossless        10           E..V....... 
#     losslesshp      11           E..V....... 
print("输出路径: {0}".format(output_pth)
print("x264 x265参数设置:")
print("crf: {0}".format(crf))
print("preset: {0}".format(preset))

print("nvenc参数设置:")
print("cq: {0}".format(cq))
print("preset: {0}".format(preset_nvenc))

16
slower
/content/Real-ESRGAN/output/output.mkv


# 264 

In [ ]:
ffmpeg

colorprim=bt709:transfer=bt709:colormatrix=bt709

-preset:v [ultrafast-placebo] -crf=[-12-51] -pix_fmt [ yuv420p / yuv420p10 ]


libx264 8bit

ffmpeg -i [.png] -c:v libx264 -preset:v [ultrafast-placebo] -profile:v high -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf=[-12-51] -x264-params "keyint=200:min-keyint=1:bframes=6:b-adapt=2:no-open-gop=1:ref=8:deblock='-1:-1':rc-lookahead=50:chroma-qp-offset=-2:aq-mode=1:aq-strength=0.8:qcomp=0.75:me=umh:merange=24:subme=10:psy-rd='1:0.1':colorprim=bt709:transfer=bt709:colormatrix=bt709" -pix_fmt yuv420p [.mp4]

libx264 10bit

ffmpeg -i [.png] -c:v libx264 -preset:v [ultrafast-placebo] -profile:v high10 -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf=[-12-51] -x264-params "keyint=200:min-keyint=1:bframes=6:b-adapt=2:no-open-gop=1:ref=8:deblock='-1:-1':rc-lookahead=50:chroma-qp-offset=-2:aq-mode=1:aq-strength=0.8:qcomp=0.75:me=umh:merange=24:subme=10:psy-rd='1:0.1':colorprim=bt709:transfer=bt709:colormatrix=bt709" -pix_fmt yuv420p10 [.mp4]

In [ ]:
%shell ffmpeg -h encoder=libx264 

In [ ]:
#264 8bit
%shell ffmpeg -r {fps} -i /content/Real-ESRGAN/results/%08d_out.png -c:v libx264 -preset:v {preset} -profile:v high -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf {crf} -x264-params keyint=200:min-keyint=1:bframes=6:b-adapt=2:no-open-gop=1:ref=8:deblock='-1:-1':rc-lookahead=50:chroma-qp-offset=-2:aq-mode=1:aq-strength=0.8:qcomp=0.75:me=umh:merange=24:subme=10:psy-rd='1:0.1':colorprim=bt709:transfer=bt709:colormatrix=bt709 -pix_fmt yuv420p {output_pth}

In [ ]:
#264 10bit（待修复
%shell ffmpeg -r {fps} -i /content/Real-ESRGAN/results/%08d_out.png -c:v libx264 -preset:v {preset} -profile:v high10 -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf {crf} -x264-params keyint=200:min-keyint=1:bframes=6:b-adapt=2:no-open-gop=1:ref=8:deblock='-1:-1':rc-lookahead=50:chroma-qp-offset=-2:aq-mode=1:aq-strength=0.8:qcomp=0.75:me=umh:merange=24:subme=10:psy-rd='1:0.1':colorprim=bt709:transfer=bt709:colormatrix=bt709 -pix_fmt yuv420p10 {output_pth}

# 265

In [ ]:
ffmpeg

range=limited:colorprim=1:transfer=1:colormatrix=1

-preset:v [ultrafast-placebo] crf=[0-51] -pix_fmt [ yuv420p / yuv420p10 ]
ultrafast superfast veryfast faster fast medium (default) slow slower veryslow placebo
master-display="G(15000,30000)B(7500,3000)R(32000,16500)WP(15635,16450)L(10000000,50)":max-cll="1000,100"


libx265 8bit

ffmpeg -i [.png] -c:v libx265 -preset:v [ultrafast-placebo] -profile:v main -x265-params "crf=[0-51]:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1" -pix_fmt yuv420p [.mp4]

libx265 10bit

ffmpeg -i [.png] -c:v libx265 -preset:v [ultrafast-placebo] -profile:v main10 -x265-params "crf=[0-51]:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1" -pix_fmt yuv420p10 [.mp4]

libx265 10bit HDR10+

ffmpeg -i [.png] -c:v libx265 -preset:v [ultrafast-placebo] -profile:v main10 -x265-params "crf=[0-51]:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=9:transfer=16:colormatrix=9:master-display="G(15000,30000)B(7500,3000)R(32000,16500)WP(15635,16450)L(10000000,50)":max-cll="1000,100":hdr10-opt=1:repeat-headers=1" -pix_fmt yuv420p10 [.mp4]

In [ ]:
%shell ffmpeg -h encoder=libx265

In [47]:
#libx265 8bit
%shell ffmpeg -r {fps} -i /content/Real-ESRGAN/results/%08d_out.png -c:v libx265 -preset:v {preset} -profile:v main -x265-params crf=16:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1 -pix_fmt yuv420p {output_pth}

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fc0a39ff090>> (for pre_run_cell):


Exception: ignored

ffmpeg version 4.4-1ubuntu0~18.04.sav0.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version='1ubuntu0~18.04.sav0.1' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fc0a39ff090>> (for post_run_cell):


Exception: ignored

In [ ]:
#libx265 10bit
%shell ffmpeg -r {fps} -i /content/Real-ESRGAN/results/%08d_out.png -c:v libx265 -preset:v {preset} -profile:v main10 -x265-params crf=16:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1 -pix_fmt yuv420p10 {output_pth}

In [ ]:
#libx265 10bit HDR10+ 
%shell ffmpeg -r {fps} -i /content/Real-ESRGAN/results/%08d_out.png -c:v libx265 -preset:v {preset} -profile:v main10 -x265-params crf=16:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=9:transfer=16:colormatrix=9:master-display="G(15000,30000)B(7500,3000)R(32000,16500)WP(15635,16450)L(10000000,50)":max-cll="1000,100":hdr10-opt=1:repeat-headers=1 -pix_fmt yuv420p10 {output_pth}

# NVENC (待修复


In [ ]:
ffmpeg

-color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv

-preset:v [12-18 (fastest-slowest)] -cq [1-51] -pix_fmt [ yuv420p / p010le ]


h264_nvenc 8bit

ffmpeg -i [.png] -flags "cgop" -g 200 -bf 4 -i_qfactor 0.71 -b_qfactor 1.3 -keyint_min 1 -c:v h264_nvenc -preset:v [12-18 (fastest-slowest)] -tune:v 1 -profile:v 2 -cq [1-51] -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -temporal-aq 1 -coder 1 -b_ref_mode 2 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt yuv420p 264.mp4

hevc_nvenc 8bit

ffmpeg -i [.png] -flags "cgop" -g 200 -i_qfactor 0.71 -keyint_min 1 -c:v hevc_nvenc -preset:v [12-18 (fastest-slowest)] -tune:v 1 -profile:v 0 -cq [1-51] -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt yuv420p 265-8.mp4

-bf 0（5代）
-bf 0 -weighted_pred 1 （6代）
-bf 5 -b_qfactor 1.3 -temporal-aq 1 -b_ref_mode 2 （7代）

hevc_nvenc 10bit

ffmpeg -i [.png] -flags "cgop" -g 200 -i_qfactor 0.71 -keyint_min 1 -c:v hevc_nvenc -preset:v [12-18 (fastest-slowest)] -tune:v 1 -profile:v 1 -cq [1-51] -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt p010le 265-10.mp4

-bf 0 -weighted_pred 1 （6代）
-bf 5 -b_qfactor 1.3 -temporal-aq 1 -b_ref_mode 2 （7代）

In [ ]:
%shell ffmpeg -h encoder=hevc_nvenc

In [ ]:
!ffmpeg -codecs | grep cuvid

In [ ]:
%shell ffmpeg -hwaccel_output_format cuda -r {fps} -f image2 -i /content/Real-ESRGAN/results/%08d_out.png -c:v hevc_nvenc {output_pth} -y

In [ ]:
#h264_nvenc 8bit
%shell ffmpeg -hwaccel_output_format cuda -r {fps} -i /content/Real-ESRGAN/results/%08d_out.png -flags "cgop" -g 200 -bf 4 -i_qfactor 0.71 -b_qfactor 1.3 -keyint_min 1 -c:v h264_nvenc -preset:v {preset_nvenc} -tune:v 1 -profile:v 2 -cq {cq} -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -temporal-aq 1 -coder 1 -b_ref_mode 2 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt yuv420p {output_pth}

In [ ]:
#hevc_nvenc 8bit
%shell ffmpeg -hwaccel_output_format cuda -r {fps} -i /content/Real-ESRGAN/results/%08d_out.png -flags "cgop" -g 200 -i_qfactor 0.71 -keyint_min 1 -c:v hevc_nvenc -preset:v {preset_nvenc} -tune:v 1 -profile:v 0 -cq {cq} -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt yuv420p {output_pth}

In [ ]:
#hevc_nvenc 10bit
%shell ffmpeg -r {fps} -i /content/Real-ESRGAN/results/%08d_out.png -flags "cgop" -g 200 -i_qfactor 0.71 -keyint_min 1 -c:v hevc_nvenc -preset:v {preset_nvenc} -tune:v 1 -profile:v 1 -cq {cq} -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt p010le {output_pth}

In [ ]:
# Download the results 网页打包下载
zip_filename = 'Real-ESRGAN_output.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} output/*")
files.download(zip_filename)

In [ ]:
#输出上传Drive
dst_path2 = os.path.join("/content/Real-ESRGAN/output", filename_out)
shutil.copy("/content/drive/MyDrive/",dst_path2)